In [17]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
import spacy
from nltk.corpus import stopwords

In [3]:
train = pd.read_csv('./data/train.csv') 


In [4]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
train.index = train['id']
x_train = train['comment_text']
y_train = train.iloc[:,2:]

In [10]:
# if the entry is clean then the sum of the values of 
#toxic, severe_toxic, etc will be >=1 so the subtraction will
#be <1

y_train['clean'] = 1 - y_train.sum(axis=1) >= 1

In [18]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner'])
stops = stopwords.words('english')

In [19]:
def normalize(comment, lowercase, remove_stops):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stops or (remove_stops and lemma not in stops):
                lemmatized.append(lemma)
    return ' '.join(lemmatized)

In [20]:
x_train_lemmatized = x_train.apply( 
    normalize, 
    lowercase=True, 
    remove_stops=True
)

/opt/miniconda3/envs/mlcysec/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [22]:
from collections import Counter
word_counts = dict()

for kind in y_train.columns:
    word_counts[kind] = Counter()
    comments = x_train_lemmatized[y_train[kind]==1]
    for _, comment in comments.iteritems():
        word_counts[kind].update(comment.split(" "))



In [23]:
import json
out_file = open('word_counts.json', 'w+')
json.dump(word_counts, out_file)

{'toxic': Counter({'cocksucker': 549,
          'piss': 172,
          'around': 351,
          'work': 497,
          'hey': 635,
          '...': 2103,
          '..': 485,
          '@': 33,
          '|': 198,
          'talk': 1364,
          '.': 35500,
          'exclusive': 9,
          'group': 84,
          'wp': 244,
          'talibans': 2,
          'good': 789,
          'destroying': 19,
          ',': 27939,
          'self': 255,
          '-': 5564,
          'appointed': 17,
          'purist': 1,
          'gang': 29,
          'one': 1524,
          'asks': 9,
          'questions': 49,
          'abt': 1,
          'anti': 237,
          'social': 42,
          'destructive': 11,
          '(': 2619,
          'non)-contribution': 1,
          '?': 8648,
          'ask': 180,
          'sityush': 2,
          'clean': 55,
          'behavior': 94,
          'issue': 90,
          'nonsensical': 6,
          'warnings': 76,
          'bye': 72,
          '!': 46273